In [1]:
from string import ascii_letters
from itertools import product
from math import inf
from heapq import heappush, heappop

In [2]:
with open("input", "r") as f:
    height_map_raw = [list(line.strip()) for line in f.readlines()]

In [3]:
for i, line in enumerate(height_map_raw):
    if 'S' in line:
        start = (i, line.index('S'))
    if 'E' in line:
        end = (i, line.index('E'))

In [4]:
heights = ascii_letters[:len(ascii_letters) // 2]
height_map = [[0 if char == 'S' else
               25 if char == 'E' else
               heights.index(char)
               for char in row] 
              for row in height_map_raw]

In [5]:
def reconstruct_path(came_from, current):
    path = [current]

    while current in came_from:
        current = came_from[current]
        path = [current] + path
    
    return path

In [6]:
def is_within_bounds(point, bounds):
    x, y = point
    x_bounds, y_bounds = bounds
    return (x_bounds[0] <= x <= x_bounds[1]
            and y_bounds[0] <= y <= y_bounds[1])

In [7]:
def h(point_a, point_b):
    return sum(abs(coord_a - coord_b)
               for coord_a, coord_b in zip(point_a, point_b))

In [8]:
def a_star(start, goal, matrix, h):

    came_from = {}
    open_set = []
    g_score = {start: 0}
    
    f_score = {start: h(start, goal)}
    
    heappush(open_set, (f_score[start], start))
    
    bounds = ((0, len(matrix) - 1),
              (0, len(matrix[0]) - 1))
    
    x, y = start
    
    while len(open_set) > 0:
        _, current = heappop(open_set)
        x, y = current
        
        if current == goal:
            return reconstruct_path(came_from, current)
        
        neighbors = [(x + x0, y + y0) for x0, y0 in product(range(-1, 2), repeat=2)
                     if abs(x0 + y0) == 1
                     and is_within_bounds((x + x0, y + y0), bounds)
                     and matrix[x + x0][y + y0] <= matrix[x][y] + 1] 
        
        for neighbor in neighbors:
            tentative_g_score = g_score.get(current, inf) + 1

            if tentative_g_score < g_score.get(neighbor, inf):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + h(neighbor, goal)
                if (f_score[neighbor], neighbor) not in open_set:
                    heappush(open_set, (f_score[neighbor], neighbor))
    return -1

### Part 1

In [9]:
len(a_star(start, end, height_map, h)) - 1

383

### Part 2

In [10]:
paths = [a_star(_start, end, height_map, h)
         for _start in [(i, j)
                        for i in range(len(height_map))
                        for j in range(len(height_map[0]))
                        if height_map[i][j] == 0]]
sorted(len(path) - 1 for path in paths if path != -1)[0]

377